In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, GRU
from tensorflow.keras.layers import Embedding
import pandas as pd

In [9]:
rbData = pd.read_parquet("rbDataV2.parquet")
wrData = pd.read_parquet("wrDataV2.parquet")
teData = pd.read_parquet("teDataV2.parquet")
rbData['position'] = [0 for i in range(rbData.shape[0])]
wrData['position'] = [1 for i in range(wrData.shape[0])]
teData['position'] = [2 for i in range(teData.shape[0])]
df = pd.concat([rbData, wrData, teData])

In [10]:
tier = []
#thresholds = [18, 14, 10, df['fantasy_points_ppr_per_game_future'].min()]
thresholds = [12, 24, 36, 48, df['fantasy_rank_future'].max()]
for row in df.itertuples():
    myBool = False
    for i in range(len(thresholds)):
        #if row.fantasy_points_ppr_per_game_future >= thresholds[i]:
        if row.fantasy_rank_future <= thresholds[i]:
            myBool = True
            tier.append(i)
            break
    if not myBool:
        tier.append(None)
df['tier_future'] = tier
nextYear = df[df.season == 2021].copy()
df = df.dropna(subset=['fantasy_points_ppr_future'])

In [11]:
df

,player_id,season,age,height,weight,experience,draft_pick,games,carries,rushing_yards,...,receiving_tds_per_game_n-2,receiving_air_yards_per_game_n-2,receiving_yards_after_catch_per_game_n-2,receiving_first_downs_per_game_n-2,receiving_epa_per_game_n-2,fantasy_points_ppr_per_game_n-2,fantasy_rank_n-2,per_game_fantasy_rank_n-2,position,tier_future
0,00-0026213,2013,27,71.0,199.0,5.0,10.0,15,259,1287.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0
1,00-0026184,2013,28,74.0,218.0,5.0,13.0,16,289,1339.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0
2,00-0027029,2013,25,71.0,210.0,4.0,21.0,16,314,1607.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0
3,00-0028198,2013,26,72.0,225.0,2.0,NaN,2,44,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1.0
4,00-0026988,2013,26,71.0,218.0,4.0,12.0,16,241,1038.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,00-0034193,2020,25,76.0,250.0,2.0,NaN,1,0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,4.0
123,00-0036250,2020,23,76.0,250.0,0.0,27.0,1,0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,4.0
125,00-0033611,2020,25,77.0,243.0,3.0,NaN,1,0,0.0,...,0.066667,45.333333,6.466667,0.933333,-1.758964,4.953333,30.0,46.0,2,2.0
126,00-0033421,2020,27,78.0,256.0,3.0,NaN,1,0,0.0,...,0.000000,6.666667,1.333333,0.166667,0.126974,1.966667,92.0,97.0,2,4.0
